In [1]:
import numpy as np
import pandas as pd
from seaborn import regplot
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/steam.csv")

In [3]:
df.english = df.english.astype('object')
df.required_age = df.required_age.astype('object')

X = df.drop(['appid','owners','positive_ratings','negative_ratings','name','developer','publisher'],axis=1)
X['positive_percentage'] = df.positive_ratings/(df.positive_ratings+df.negative_ratings) 


print(X.dtypes)
X.head(3)

release_date            object
english                 object
platforms               object
required_age            object
categories              object
genres                  object
steamspy_tags           object
achievements             int64
average_playtime         int64
median_playtime          int64
price                  float64
positive_percentage    float64
dtype: object


,release_date,english,platforms,required_age,categories,genres,steamspy_tags,achievements,average_playtime,median_playtime,price,positive_percentage
0,2000-11-01,1,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,17612,317,7.19,0.973888
1,1999-04-01,1,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,277,62,3.99,0.839787
2,2003-05-01,1,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,187,34,3.99,0.895648


In [4]:
# X_number = X.select_dtypes(exclude='object')
# X_number.head(3)

In [5]:
X_number = X.select_dtypes(exclude='object')
X_category = X.select_dtypes(include='object')
for k in X_category.columns:
    X_category[k] = X_category[k].astype('category')

X_category.head(3)


,release_date,english,platforms,required_age,categories,genres,steamspy_tags
0,2000-11-01,1,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
1,1999-04-01,1,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
2,2003-05-01,1,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer


In [6]:
print(X_category.shape)
print(X_category.dtypes)

(27075, 7)
release_date     category
english          category
platforms        category
required_age     category
categories       category
genres           category
steamspy_tags    category
dtype: object


In [7]:
engDummy = pd.get_dummies(X_category.iloc[:,[1,3]]) 
engDummy.head(3)

,english_0,english_1,required_age_0,required_age_3,required_age_7,required_age_12,required_age_16,required_age_18
0,0,1,1,0,0,0,0,0
1,0,1,1,0,0,0,0,0
2,0,1,1,0,0,0,0,0


In [8]:
X_category['release_date'] = X_category['release_date'].astype('datetime64')

In [9]:
print(X_category.shape)

(27075, 7)


In [10]:
X_category['release_date'].sort_values(ascending=True)

2685    1997-06-30
6       1998-11-08
1       1999-04-01
4       1999-11-01
0       2000-11-01
           ...    
27060   2019-04-25
26695   2019-04-25
22522   2019-04-25
24430   2019-04-26
23726   2019-05-01
Name: release_date, Length: 27075, dtype: datetime64[ns]

In [11]:
X_category['release_date']  = (X_category['release_date'] - pd.to_datetime('1997-06-30')).dt.total_seconds()/86400

In [12]:
X_category.head(3)

,release_date,english,platforms,required_age,categories,genres,steamspy_tags
0,1220.0,1,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
1,640.0,1,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer
2,2131.0,1,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer


In [13]:
platDummy = X_category['platforms'].str.get_dummies(sep=';')

In [14]:
cateDummy = X_category['categories'].str.get_dummies(sep=';')

In [15]:
genDummy = X_category['genres'].str.get_dummies(sep=';')

In [16]:
tagDummy = X_category['steamspy_tags'].str.get_dummies(sep=';').add_prefix('tag_')

In [17]:
X_number = pd.concat([X_category.release_date,X_number],axis=1)
stnd = StandardScaler()
stnd.fit(X_number)
X_number = pd.DataFrame(stnd.transform(X_number),
                                columns=X_number.columns)
X_number.head(3)

,release_date,achievements,average_playtime,median_playtime,price,positive_percentage
0,-7.584710,-0.128306,9.557829,0.072624,0.141186,1.110537
1,-8.329745,-0.128306,0.069619,-0.035710,-0.265175,0.536451
2,-6.414491,-0.128306,0.020358,-0.047606,-0.265175,0.775589


In [18]:
X_category = pd.concat([engDummy,platDummy,cateDummy,genDummy,tagDummy],axis=1)

In [19]:
X_category.head(3)

,english_0,english_1,required_age_0,required_age_3,required_age_7,required_age_12,required_age_16,required_age_18,linux,mac,...,tag_Warhammer 40K,tag_Web Publishing,tag_Werewolves,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports
0,0,1,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,0


In [20]:
df.owners=df.owners.astype("string")
owners=df.owners.str.split("-")

result=[]
for i in range(len(owners)):
    low=int(owners[i][0])
    high=int(owners[i][1])
    result.append(high)

df["Owners"]=result

df["popularity"]=pd.cut(df["Owners"],bins=[0,20000,100000, 200000000], labels=["Low popularity","Average", "Popular"])
df["popularity"].value_counts()

df["popularity"].isna().value_counts()

False    27075
Name: popularity, dtype: int64

In [21]:
X = pd.concat([X_number,X_category,df.popularity,df.positive_ratings, df.negative_ratings],axis=1)
X.rename(columns = {'release_date':'day_elapse'}, inplace = True)
print(X.shape)
X.head(3)

(27075, 417)


,day_elapse,achievements,average_playtime,median_playtime,price,positive_percentage,english_0,english_1,required_age_0,required_age_3,...,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,popularity,positive_ratings,negative_ratings
0,-7.584710,-0.128306,9.557829,0.072624,0.141186,1.110537,0,1,1,0,...,0,0,0,0,0,0,0,Popular,124534,3339
1,-8.329745,-0.128306,0.069619,-0.035710,-0.265175,0.536451,0,1,1,0,...,0,0,0,0,0,0,0,Popular,3318,633
2,-6.414491,-0.128306,0.020358,-0.047606,-0.265175,0.775589,0,1,1,0,...,0,0,0,1,0,0,0,Popular,3416,398


In [23]:
X.to_csv('project_preprocess.csv',index=False)